In [1]:
import pandas as pd


from tqdm import tqdm
import datetime


from AvailableErrors import *
from Andrii.Dir_Task_JAN_FEB.tools_functions import createTimeDifference

In [2]:
global _WEEK_DICT, _HOLIDAY
_WEEK_DICT = {0:'ПН', 1:'ВТ', 2:'СР', 3:'ЧТ', 4:'ПТ', 5:'СБ', 6:'ВС'}
_HOLIDAY_MARKERS = [5,6]

In [62]:
class StrategyExtractor:
    @classmethod
    def ExtractorIndex(cls):
        return 'StrategyExtractor'

    def __init__(self):
        self.compatible = [HubTrendFollowingStrategy.StrategyIndex()]

        self._start = None
        self._end = None
        self.input_data = None

        self.filters = list()
        self.after_filters = list()

        pass

    def input_params(self, start_horizon:datetime.timedelta, end_horizon: datetime.timedelta):
        self._start = start_horizon
        self._end = end_horizon

    def apply_filters(self, single_filter):
        if self.ExtractorIndex not in single_filter.compatible:
            raise StrategyErrors().UnCompatibleFilter(Strategy=self, Filter=single_filter)
        self.filters.append(single_filter)

    def transfer_data(self, data: pd.DataFrame):
        if type(data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.input_data = data

    def making_horizon(self):
        for dot_on_all_data in tqdm(self.input_data.index):
            logical_filter = True
            #   SELECT BORDERS
            buffer_data = self.input_data.loc[dot_on_all_data + self._start: dot_on_all_data + self._end]
            for single_filter in self.filters:
                FILTER = single_filter
                FILTER.apply_buffer_data(buffer_data=buffer_data)
                if not FILTER.condition():
                    logical_filter = False
                    break


            if logical_filter:
                self.after_filters.append(buffer_data)
                break   #   This break scares me

"""================================================================================================================================================="""
class CentralDotFilter:
    def __init__(self):
        self.compatible = [StrategyExtractor.ExtractorIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'CentralDotFilter'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = buffer_data.copy()

    def condition(self, CentralDotCoords):
        pass

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class ConsistingTime:
    def __init__(self,*consist_time):
        self.compatible = [StrategyExtractor.ExtractorIndex]
        self.buffer_data = None
        for _ in consist_time:
            if type(_) != pd._libs.tslibs.timedeltas.Timedelta:
                raise FilterErrors().WrongShiftedType()
        self.consist_time = [_ for _ in consist_time]

    @classmethod
    def FilterIndex(cls):
        return 'ConsistingTime'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        # if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
        #     raise StrategyErrors().WrongIndicesType()
        #
        # ZERO_TIME = buffer_data.index[0].to_numpy()
        # ZERO_TIME = ZERO_TIME.astype('datetime64[s]').item().date()
        # self.buffer_data = buffer_data.index - pd.to_datetime(ZERO_TIME)

        # Если не использовать вызов функции - будет быстрей. Но так зависимости выглядят понятней
        self.buffer_data = createTimeDifference(buffer_data.index)


        return None

    def condition(self):
        for timedelta in self.consist_time:
            #if not timedelta in self.buffer_data.index:
            if not timedelta in self.buffer_data:
                return False

        return True

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class NoHolidays:
    def __init__(self):
        self.compatible = [StrategyExtractor.ExtractorIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'NoHolidays'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = buffer_data.copy()
        return None

    def condition(self):
        for holiday in self.buffer_data.index.dayofweek:
            if holiday in _HOLIDAY_MARKERS:
                return False

        return True

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class LeftBorder:
    def __init__(self, left_border:pd._libs.tslibs.timedeltas.Timedelta):
        self.compatible = [StrategyExtractor.ExtractorIndex]

        if type(left_border) != pd._libs.tslibs.timedeltas.Timedelta:
            raise FilterErrors().WrongShiftedType()
        self.buffer_data = None
        self._left_border = left_border

    @classmethod
    def FilterIndex(cls):
        return 'LeftBorder'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = createTimeDifference(buffer_data.index)
        return None

    def condition(self):
        if self.buffer_data[0] != self._left_border:
            return False

        return True

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""

class RightBorder:
    def __init__(self, right_border:pd._libs.tslibs.timedeltas.Timedelta):
        self.compatible = [StrategyExtractor.ExtractorIndex]

        if type(right_border) != pd._libs.tslibs.timedeltas.Timedelta:
            raise FilterErrors().WrongShiftedType()
        self.buffer_data = None
        self._right_border = right_border

    @classmethod
    def FilterIndex(cls):
        return 'RightBorder'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = createTimeDifference(buffer_data.index)
        return None

    def condition(self):
        if self.buffer_data[0] != self._right_border:
            return False

        return True

"""================================================================================================================================================="""

class HubTrendFollowingStrategy:
    @classmethod
    def StrategyIndex(cls):
        return 'HubTrendFollowingStrategy'
    def __init__(self):
        self.all_data = None
        self.central_dot_position = None
        self.lookback = None
        self.potential_hold = None
        self.extractor = None

    def add_extractor(self, method):
        if self.StrategyIndex() not in method.compatible:
            raise StrategyErrors().UnCompatibleExtractor(HubStrategy=self, StrategyExtractor=method)
        self.extractor = method

    def transfer_data(self, raw_data:pd.DataFrame):
        if type(raw_data) != pd.DataFrame:
            raise TypeError('Input Data must have type pandas DataFrame')
        self.all_data = raw_data.copy()

    def create_lookback_center(self, splitDot_position:int):
        if type(splitDot_position) != int:
            raise SplitDataErrors().WrongCentralDotTypeError()

        self.central_dot_position = splitDot_position

    def testing_simulation(self):
        self.extractor.transfer_data(self.all_data)
        self.extractor.making_horizon()
        return self.extractor.after_filters

    def add_Rules_constructor(self, rulesConstructor):
        if self.StrategyIndex not in rulesConstructor.compatible:
            raise StrategyErrors().UnCompatibleRulesConstructor(HubStrategy=self, RulesConstructor=rulesConstructor)
"""================================================================================================================================================="""

class TrendFollowingStrategyRulesConstructor:
    def __init__(self):
        self.compatible = [HubTrendFollowingStrategy.StrategyIndex]

        self.opening_rules = list()
        self.holding_rules = list()

    @classmethod
    def StrategyRuleConstructorType(cls):
        return 'TrendFollowConstructor'

    def add_opening_rule(self, opening_rule):
        if self.StrategyRuleConstructorType not in opening_rule.compatible:
            raise RulesConstructorErrors().UnCompatibleComplexRule(RuleConstructor=self, ComplexRule=opening_rule)
        self.opening_rules.append(opening_rule)

    def add_hold_rule(self, hold_rule):
        if self.StrategyRuleConstructorType not in hold_rule.compatible:
            raise RulesConstructorErrors().UnCompatibleComplexRule(RuleConstructor=self, ComplexRule=hold_rule)
        self.opening_rules.append(hold_rule)

"""================================================================================================================================================="""

class OpeningCrossHighLowRule:
    def __init__(self):
        self.compatible = [TrendFollowingStrategyRulesConstructor.StrategyRuleConstructorType]

        self.low = None
        self.high = None

    @classmethod
    def RuleIndex(cls):
        return 'OpeningCrossHighLowRule'

    def add_low_calculation(self, low):
        if self.RuleIndex not in low.compatible:
            raise SmallWorkers().UnCompatibleRuleBlock(ComplexRule=self, Worker=low)
        self.low = low

    def add_high_calculation(self, high):
        self.high = high



"""================================================================================================================================================="""

class CalculateLow:
    def __init__(self):
        self.compatible = [OpeningCrossHighLowRule.RuleIndex]

    @classmethod
    def WorkerIndex(self):
        return 'CalculateLow'

    def testing(self):
        print('LOWER')

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""

class CalculateHigh:
    def __init__(self):
        self.compatible = [OpeningCrossHighLowRule.RuleIndex]

    @classmethod
    def WorkerIndex(self):
        return 'CalculateHigh'
    def testing(self):
        print('HIGHER')
"""================================================================================================================================================="""

"""
horizon = StrategyExtractor()
constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)
nodolidays = NoHolidays()
horizon.apply_filters(nodolidays)
horizon.transfer_data(d.iloc[:1000])
horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
horizon.after_filters
"""

r = CrossHighLowRule()
r.RuleIndex()
r.add_low_calculation(CalculateLow())
r.add_high_calculation(CalculateHigh())

In [ ]:
d = pd.read_csv('zip_data/AUDCAD.csv', index_col=0)
d.index = pd.to_datetime(d.index)
d = d.resample('1T').first()


In [49]:
horizon = StrategyExtractor()

constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)

nodolidays = NoHolidays()
horizon.apply_filters(nodolidays)

leftBorder = LeftBorder(pd.Timedelta(hours=15))
horizon.apply_filters(leftBorder)

horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))

,open,high,low,close
time,,,,
2018-11-01 00:00:00,0.931100,0.931135,0.930670,0.931025
2018-11-01 01:00:00,0.931275,0.931330,0.930935,0.931175
2018-11-01 02:00:00,0.931645,0.931725,0.931540,0.931690
2018-11-01 03:00:00,0.931975,0.932060,0.931935,0.931935
2018-11-01 04:00:00,0.934755,0.935005,0.934565,0.934570
...,...,...,...,...
2021-11-12 20:00:00,0.919545,0.919645,0.919530,0.919600
2021-11-12 21:00:00,0.920485,0.920535,0.920420,0.920475
2021-11-12 22:00:00,0.920465,0.920510,0.920415,0.920415


In [50]:
TF = HubTrendFollowingStrategy()
TF.add_extractor(horizon)
TF.extractor.after_filters.clear()
TF.transfer_data(d)
TF.testing_simulation()

  0%|          | 1605/1595565 [00:00<11:50, 2242.00it/s]


[                         open      high       low     close
 time                                                       
 2018-11-01 15:00:00  0.939375  0.939760  0.939285  0.939585
 2018-11-01 15:01:00  0.939585  0.939610  0.939380  0.939450
 2018-11-01 15:02:00  0.939450  0.939505  0.939360  0.939440
 2018-11-01 15:03:00  0.939440  0.939575  0.939370  0.939575
 2018-11-01 15:04:00  0.939575  0.939675  0.939545  0.939640
 ...                       ...       ...       ...       ...
 2018-11-02 21:56:00  0.943025  0.943135  0.942845  0.942860
 2018-11-02 21:57:00  0.942860  0.942870  0.942730  0.942835
 2018-11-02 21:58:00  0.942835  0.942910  0.942780  0.942880
 2018-11-02 21:59:00  0.942880  0.942960  0.942810  0.942945
 2018-11-02 22:00:00  0.942945  0.943245  0.942945  0.943105
 
 [1861 rows x 4 columns]]

In [ ]:
horizon = StrategyExtractor()

constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)

nodolidays = NoHolidays()
horizon.apply_filters(nodolidays)

leftBorder = LeftBorder(pd.Timedelta(hours=15))
horizon.apply_filters(leftBorder)

horizon.transfer_data(d)

horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
#horizon.after_filters

TF = HubTrendFollowingStrategy()
TF.extract_data_method(horizon)

In [11]:
horizon.after_filters

[                         open      high       low     close
 time                                                       
 2018-11-01 15:00:00  0.939375  0.939760  0.939285  0.939585
 2018-11-01 16:00:00  0.939495  0.939565  0.939375  0.939445
 2018-11-01 17:00:00  0.939295  0.939895  0.939270  0.939865
 2018-11-01 18:00:00  0.942600  0.942680  0.942535  0.942640
 2018-11-01 19:00:00  0.942275  0.942335  0.942075  0.942140
 2018-11-01 20:00:00  0.942205  0.942270  0.942025  0.942195
 2018-11-01 21:00:00  0.942930  0.943070  0.942895  0.943030
 2018-11-01 22:00:00  0.943120  0.943165  0.943050  0.943065
 2018-11-01 23:00:00  0.943680  0.943690  0.943505  0.943515
 2018-11-02 00:00:00  0.942955  0.943045  0.942955  0.943040
 2018-11-02 01:00:00  0.942280  0.942380  0.942265  0.942335
 2018-11-02 02:00:00  0.942200  0.942445  0.942175  0.942385
 2018-11-02 03:00:00  0.943185  0.943295  0.943160  0.943205
 2018-11-02 04:00:00  0.943615  0.943615  0.943210  0.943225
 2018-11-02 05:00:00  0.

In [ ]:
"""d = pd.read_csv('zip_data/AUDCAD.csv', index_col=0)
d.index = pd.to_datetime(d.index)
d = d.resample('1H').first()
d = d.iloc[:150_0]
horizon = StrategyExtractor()
constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)
#nodolidays = NoHolidays()
#horizon.apply_filters(nodolidays)
horizon.transfer_data(d)
horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
horizon.after_filters"""

'<__main__.TrendFollowingStrategy object at 0x11ec65f70>'